# On codera la lecture d'ARN

In [3]:
DEBUT_LECTURE = "AUG"
STOPS = ["UAA", "UAG", "UGA"]

acide_amine = {
    "UUU" : "Phe",
    "UUC" : "Phe",

    "UUA" : "Leu",
    "UUG" : "Leu",
    "CUU" : "Leu",
    "CUC" : "Leu",
    "CUA" : "Leu",
    "CUG" : "Leu",

    "AUU" : "Ile",
    "AUC" : "Ile",
    "AUA" : "Ile",

    "AUG" : "Met",

    "GUU" : "Val",
    "GUC" : "Val",
    "GUA" : "Val",
    "GUG" : "Val",

    "UCU" : "Ser",
    "UCC" : "Ser",
    "UCA" : "Ser",
    "UCG" : "Ser",

    "CCU" : "Pro",
    "CCC" : "Pro",
    "CCG" : "Pro",
    "CCA" : "Pro",

    "ACU" : "Thr",
    "ACA" : "Thr",
    "ACG" : "Thr",
    "ACC" : "Thr",

    "GCU" : "Ala",
    "GCA" : "Ala",
    "GCC" : "Ala",
    "GCG" : "Ala",

    "UAU" : "Tyr",
    "UAC" : "Tyr",

    "CAU" : "His",
    "CAC" : "His",

    "CAA" : "Gln",
    "CAG" : "Gln",

    "AAU" : "Asn",
    "AAC" : "Asn",

    "AAA" : "Lys",
    "AAG" : "Lys",

    "GAU" : "Asp",
    "GAC" : "Asp",

    "GAA" : "Glu",
    "GAG" : "Glu",

    "UGU" : "Cys",
    "UGC" : "Cys",

    "UGG" : "Trp",

    "CGU" : "Arg",
    "CGA" : "Arg",
    "CGC" : "Arg",
    "CGG" : "Arg",
    "AGA" : "Arg",
    "AGG" : "Arg",

    "AGU" : "Ser",
    "AGC" : "Ser",

    "GGU" : "Gly",
    "GGC" : "Gly",
    "GGA" : "Gly",
    "GGG" : "Gly"

}


In [ ]:
# Trouver le début de la séquence
def find_start(arn):
    if arn[0:3] == DEBUT_LECTURE:
        return arn[3:]
    return find_start(arn[1:])


# Lire la séquence après avoir enlevé les éléments qu'on lit pas
def read_sequence(arn):
    if len(arn) <= 2 or arn[0:3] in STOPS:
        return ""
    return (acide_amine[arn[0:3]] + "-" + read_sequence(arn[3:]))

In [29]:
# On lit notre ARN
def read_arn(arn):
    return read_sequence(find_start(arn))

In [33]:
ARN = "AUGUUAGAAAUUUCAUGGUGCCUUA"

print(read_arn(ARN))

Leu-Glu-Ile-Ser-Trp-Cys-Leu-
